In [1]:
# Copyright 2019-present NAVER Corp.
# Apache License v2.0

# Wonseok Hwang
# Sep30, 2018
import os, sys, argparse, re, json,ujson
import pandas as pd
import numpy as np
from matplotlib.pylab import *
import tensorflow as tf
import random as python_random
# import torchvision.datasets as dsets
import tensorflow_hub as hub

from sqlova.utils.utils_wikisql import *
from sqlova.model.nl2sql.wikisql_models import *
from sqlnet.dbengine import DBEngine
import bert.tokenization as tokenization
from bert.modeling import BertConfig, BertModel
from tensorflow.keras import backend as K
import random

W0404 16:59:31.571887 139964001732352 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [3]:
num_CPU = 1
num_GPU = 0
num_cores = 8
config = tf.ConfigProto(intra_op_parallelism_threads=num_cores,
                    inter_op_parallelism_threads=num_cores,
                    allow_soft_placement=True,
                    device_count = {'CPU' : num_CPU,
                                    'GPU' : num_GPU}
                   )

sess = tf.Session(config = config)

In [4]:
os.environ['TFHUB_CACHE_DIR'] = '/DataDrive/master-wikisql/tfhub_cache/'

In [5]:
bert_path = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

In [50]:
#sess = tf.Session()

In [74]:
set(wc.flatten())

{-1.0, 0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0}

In [87]:
max_num_headers = len(set(wc.flatten()))
w = np.ones([wc.shape[0],wc.shape[1],max_num_headers])
w.shape
w[np.where(wc==-1)] = 0
len(w)

2000

In [16]:
training_data, train_labels = makeTrainingData(use_reduced_set=True,sample_size=2000)
sc,sa,wn,wc,wo = train_labels
max_num_headers = len(set(wc.flatten()))
w = np.ones([wc.shape[0],wc.shape[1]])
#w[np.where(w>=0)] = 1
w[np.where(wc==-1)] = 0
max_out_len = wc.shape[1]
split = int(0.8*len(wc))
test_data = training_data[split:]
training_data = training_data[:split]

test_labels_sc = sc[split:]
train_labels_sc = sc[:split]

test_labels_sa = sa[split:]
train_labels_sa = sa[:split]


test_labels_wn = wn[split:]
train_labels_wn = wn[:split]

test_labels_wc = wc[split:]
train_labels_wc = wc[:split]

test_labels_wo = wo[split:]
train_labels_wo = wo[:split]

train_weights = w[:split]
test_weights = w[split:]
#test_labels = np.array(test_labels).reshape(-1, 1)
#train_labels= np.array(train_labels).reshape(-1, 1)

train_input_ids,train_input_masks,train_segment_ids = formatForBert(training_data)
test_input_ids,test_input_masks,test_segment_ids = formatForBert(test_data)


In [134]:
test_labels_wc.shape
test_labels_wo.shape
train_labels_wo.shape

(1600, 3)

In [19]:
training_data,train_labels = makeTrainingData(use_reduced_set=True,sample_size=2000)

In [ ]:

# Build the rest of the classifier 
    
max_out_len = 3
model = build_model(max_seq_length=222,max_out_len=max_out_len)

# Instantiate variables
initialize_vars(sess)

model.fit(
    #[train_input_ids, train_input_masks, train_segment_ids, train_weights], 
    [train_input_ids, train_input_masks, train_segment_ids], 
        {"sc_output":train_labels_sc,"sa_output":train_labels_sa,
         "wn_output":train_labels_wn,"wc_output":train_labels_wc,"wo_output":train_labels_wo},
    validation_data=(#[test_input_ids,test_input_masks,test_segment_ids, test_weights],
                [test_input_ids,test_input_masks,test_segment_ids],
                     {"sc_output":test_labels_sc,"sa_output":test_labels_sa,
                      "wn_output":test_labels_wn,"wc_output":test_labels_wc,
                     "wo_output":test_labels_wo}),
    epochs=1,
    batch_size=32
)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0404 17:06:16.530830 139964001732352 saver.py:1483] Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0404 17:06:18.430361 139964001732352 saver.py:1483] Saver not created because there are no variables in the graph to restore


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          (None, 222)          0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        (None, 222)          0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        (None, 222)          0                                            
__________________________________________________________________________________________________
bert_layer_4 (BertLayer)        (None, 768)          110104890   input_ids[0][0]                  
                                                                 input_masks[0][0]                
          

W0404 17:06:26.281414 139964001732352 deprecation.py:323] From /DataDrive/master-wikisql/wsqlenv/lib/python3.6/site-packages/tensorflow/python/ops/math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.


 352/1600 [=====>........................] - ETA: 9:21 - loss: 6.8084 - sc_output_loss: 2.3744 - sa_output_loss: 1.0673 - wn_output_loss: 0.6153 - wc_output_loss: 2.4631 - wo_output_loss: 0.2884 - sc_output_acc: 0.1278 - sa_output_acc: 0.6335 - wn_output_acc: 0.8011 - wc_output_acc: 0.2159 - wo_output_acc: 0.9062

In [68]:
i = 1
test_example = test_data[i]
p_id,p_mask,p_seg = formatForBert([test_example])

input1,input2,input3,input4 = p_id,p_mask,p_seg,np.array([test_weights[i,:]])

In [70]:
test_example

{'query': ['how',
  'many',
  'end',
  'of',
  'term',
  'with',
  'age',
  'at',
  'inauguration',
  'being',
  '64-066',
  '64years',
  ',',
  '66days'],
 'header': ['#',
  'president',
  'date of birth',
  'date of inauguration',
  'age at inauguration',
  'end of term',
  'length of retirement',
  'date of death',
  'lifespan']}

In [69]:
model.predict([input1,input2,input3,input4])

[array([[0.2668911 , 0.1030051 , 0.10034642, 0.14030127, 0.17761466,
         0.08895458, 0.05044549, 0.02228669, 0.03487641, 0.00805316,
         0.0041892 , 0.00161389, 0.00142196]], dtype=float32),
 array([[0.82275784, 0.02035269, 0.05989522, 0.09699427]], dtype=float32),
 array([[0.0288064 , 0.89778435, 0.07083937, 0.00256985]], dtype=float32),
 array([[0.94555503, 0.15660135, 0.00410538]], dtype=float32),
 array([[0.95385194, 0.9670656 , 0.01036781]], dtype=float32)]

In [89]:
{"sc_output":train_labels_sc,"sa_output":train_labels_sa,
 "wn_output":train_labels_wn,"wc_output":train_labels_wc,"wo_output":train_labels_wo},
train_labels_wo.max()

2.0

In [20]:
# Build model
def weighted_categorical_crossentropy(weights):
    """
    A weighted version of keras.objectives.categorical_crossentropy
    
    Variables:
        weights: numpy array of shape (C,) where C is the number of classes
    
    Usage:
        weights = np.array([0.5,2,10]) # Class one at 0.5, class 2 twice the normal weights, class 3 10x.
        loss = weighted_categorical_crossentropy(weights)
        model.compile(loss=loss,optimizer='adam')
    """
    
    #weights = K.variable(weights)
        
    def loss(y_true, y_pred):
        # scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        # clip to prevent NaN's and Inf's
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        # calc
        loss = y_true * K.log(y_pred) * weights
        loss = -K.sum(loss, -1)
        return loss
    
    return loss

def build_model(max_seq_length,max_out_len,max_header_length=13): 
    in_id = tf.keras.layers.Input(shape=(max_seq_length,), name="input_ids")
    in_mask = tf.keras.layers.Input(shape=(max_seq_length,), name="input_masks")
    in_segment = tf.keras.layers.Input(shape=(max_seq_length,), name="segment_ids")
    #weight_input = tf.keras.layers.Input(shape=(max_out_len,), name="ce_weights")
    
    bert_inputs = [in_id, in_mask, in_segment]
    all_inputs = [in_id, in_mask, in_segment]
    bert_output = BertLayer(n_fine_tune_layers=3)(bert_inputs)

    denseSC = tf.keras.layers.Dense(256, activation='relu',name="denseSC")(bert_output)
    denseSA = tf.keras.layers.Dense(256, activation='relu',name="denseSA")(bert_output)
    denseWN = tf.keras.layers.Dense(256, activation='relu',name="denseWN")(bert_output)
    denseWC = tf.keras.layers.Dense(256, activation='relu',name="denseWC")(bert_output)
    denseWO = tf.keras.layers.Dense(256, activation='relu',name="denseWO")(bert_output)
    
    
    sc = tf.keras.layers.Dense(max_header_length, activation='softmax',name="sc_output")(denseSC)
    sa = tf.keras.layers.Dense(4, activation='softmax',name="sa_output")(denseSA)
    wn = tf.keras.layers.Dense(4, activation='softmax',name="wn_output")(denseWN)    
    wc = tf.keras.layers.Dense(max_header_length, activation='softmax',name="wc_output")(denseWC)
    wo = tf.keras.layers.Dense(3, activation='softmax',name="wo_output")(denseWO)

    #loss = weighted_categorical_crossentropy(weight_input)    
    #loss=,
    model = tf.keras.models.Model(inputs=all_inputs, outputs=[sc,sa,wn,wc,wo])
    

    model.compile(loss={"sc_output":"sparse_categorical_crossentropy",
                        "sa_output":"sparse_categorical_crossentropy",
                        "wn_output":"sparse_categorical_crossentropy",
                        #'wc_output': loss, 
                        #'wo_output': loss}, optimizer='adam', metrics=['accuracy'])
                        'wc_output': 'categorical_crossentropy', 
                        'wo_output': 'categorical_crossentropy'}, optimizer='adam', metrics=['accuracy'])
    model.summary()
    
    return model

def initialize_vars(sess):
    sess.run(tf.local_variables_initializer())
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    K.set_session(sess)


In [128]:
test_labels_wc.shape

(400, 13)

## DEFINE THE ACTUAL BERT LAYER





In [12]:
class BertLayer(tf.layers.Layer):
    def __init__(self, n_fine_tune_layers=10, **kwargs):
        self.n_fine_tune_layers = n_fine_tune_layers
        self.trainable = True
        self.output_size = 768
        super(BertLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.bert = hub.Module(
            bert_path,
            trainable=self.trainable,
            name="{}_module".format(self.name)
        )
        trainable_vars = self.bert.variables
        
        # Remove unused layers
        trainable_vars = [var for var in trainable_vars if not "/cls/" in var.name]
        
        # Select how many layers to fine tune
        trainable_vars = trainable_vars[-self.n_fine_tune_layers :]
        
        # Add to trainable weights
        for var in trainable_vars:
            self._trainable_weights.append(var)
        
        # Add non-trainable weights
        for var in self.bert.variables:
            if var not in self._trainable_weights:
                self._non_trainable_weights.append(var)
        
        super(BertLayer, self).build(input_shape)

    def call(self, inputs):
        inputs = [K.cast(x, dtype="int32") for x in inputs]
        input_ids, input_mask, segment_ids = inputs
        bert_inputs = dict(
            input_ids=input_ids, input_mask=input_mask, segment_ids=segment_ids
        )
        result = self.bert(inputs=bert_inputs, signature="tokens", as_dict=True)[
            "pooled_output"
        ]
        return result

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_size)

In [6]:
bert_module = hub.Module("https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1", trainable=True)

Instructions for updating:
Colocations handled automatically by placer.


W0403 12:35:06.081140 139717726570240 deprecation.py:323] From /DataDrive/master-wikisql/wsqlenv/lib/python3.6/site-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


In [11]:
def create_tokenizer_from_hub_module(bert_hub_module_handle):
    """Get the vocab file and casing info from the Hub module."""
    with tf.Graph().as_default():
        tf.print('GRAPH STARTED')
        bert_module = hub.Module(bert_hub_module_handle)
        tf.print('MODULE DOWNLOADED')
        tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
        tf.print('TOKENIZATION INFO LOADED')
        with tf.Session() as sess:
            tf.print('SESSION STARTED')
            vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                                tokenization_info["do_lower_case"]])
            tf.print('VARIABLES RAN')
            return FullTokenizer(
                vocab_file=vocab_file, do_lower_case=do_lower_case)

## BERT FORMATTING AND OUTPUT GENERATION

In [10]:
def generate_inputs(tokenizer, nlu1_tok, hds1):
   tokens = []
   segment_ids = []
   tokens.append("[CLS]")

   segment_ids.append(0)
   for token in nlu1_tok:
       tokens.append(token)
       segment_ids.append(0)
   tokens.append("[SEP]")
   segment_ids.append(0)

   header_break = [len(tokens),len(tokens) + len(hds1)]
   # for doc
   for i, hds11 in enumerate(hds1):
       sub_tok = tokenizer.tokenize(hds11)
       tokens += sub_tok
       segment_ids += [1] * len(sub_tok)
       if i < len(hds1)-1:
           tokens.append("[SEP]")
           segment_ids.append(0)
       elif i == len(hds1)-1:
           tokens.append("[SEP]")
           segment_ids.append(1)
       else:
           raise EnvironmentError


   return tokens, segment_ids, header_break

def _formatForBert(tokenizer, query_token, header_token, max_seq_length):
    #####For each example, tokenize with BERT tokenizer
    #Mark each example begining, and separate each header 
    #so BERT understands the query is a meaningful sequence of words but the headers are not
    #concatenate the tokens with breaks but preserve where the headers stop and start
    #then pad each example to the max sequence length within each batch
    double_tokenized_tokens = []
    for (i, token) in enumerate(query_token):
            #t_to_tt_idx1.append(
            #    len(double_tokenized_tokens))  # all_doc_tokens[ indicate the start position of original 'white-space' tokens.
            sub_tokens = tokenizer.tokenize(token)
            for sub_token in sub_tokens:
            #    tt_to_t_idx1.append(i)
                double_tokenized_tokens.append(sub_token)  # all_doc_tokens are further tokenized using WordPiece tokenizer

    tokens1, segment_ids1, header_break = generate_inputs(tokenizer, double_tokenized_tokens, header_token)
    input_ids1 = tokenizer.convert_tokens_to_ids(tokens1)

    # Input masks
    # The mask has 1 for real tokens and 0 for padding tokens. Only real
    # tokens are attended to.
    input_mask1 = [1] * len(input_ids1)

        # 3. Zero-pad up to the sequence length.
    while len(input_ids1) < max_seq_length:
            input_ids1.append(0)
            input_mask1.append(0)
            segment_ids1.append(0)

    assert len(input_ids1) == max_seq_length
    assert len(input_mask1) == max_seq_length
    assert len(segment_ids1) == max_seq_length
    return input_ids1,tokens1,segment_ids1,input_mask1,header_break

def formatForBert(train_data,BERT_PT_PATH= "/DataDrive/master-wikisql/annotated_data/",bert_type="uncased_L-12_H-768_A-12",max_seq_length=222):
    
    input_ids = []
    tokens = []
    segment_ids = []
    input_masks = []
    header_breaks = []

    bert_config_file = os.path.join(BERT_PT_PATH, f'bert_config_{bert_type}.json')
    vocab_file = os.path.join(BERT_PT_PATH, f'vocab_{bert_type}.txt')

    bert_config = BertConfig.from_json_file(bert_config_file)
    tokenizer = tokenization.FullTokenizer(
        vocab_file=vocab_file, do_lower_case=True)
    
    for train_example in train_data:
        query_token = train_example["query"]
        header_token = train_example["header"]
        input_ids1, tokens1, segment_ids1, input_mask1, header_break = \
                _formatForBert(tokenizer, query_token,header_token,max_seq_length)
        input_ids.append(input_ids1)
        tokens.append(tokens1)
        segment_ids.append(segment_ids1)
        input_masks.append(input_mask1)
        header_breaks.append(header_break)

    #TODO return header breaks and use it too
    #bert_inputs = []
    return np.array(input_ids), np.array(input_masks), np.array(segment_ids)

def getBertOutput(bert_inputs, max_seq_length):
    all_input_ids = tf.keras.layers.Input(shape=(max_seq_length,), name = "input_ids")
    all_input_mask = tf.keras.layers.Input(shape=(max_seq_length,), name = "input_masks")
    all_segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), name = "segment_ids")
    bert_output = BertLayer(n_fine_tune_layers=10)(bert_inputs)


## READING INPUT, PARSING INTO WHAT WILL BE FED TO THE MODEL

In [9]:
def makeTrainingData(wikisql_path = "/DataDrive/master-wikisql/annotated_data/",sample_size=32,use_reduced_set=False):
    train_data, train_table, dev_data, dev_table, _, _ = load_wikisql(wikisql_path,
                                                                  use_reduced_set, sample_size, no_w2i=True, no_hs_tok=True)
    
    return parseToTextLines(train_data,train_table)

def parseToTextLines(train_data,train_table):
    train_dataframe = pd.DataFrame(train_data)
    question_toks = [[x.lower() for x in sublist] for sublist in train_dataframe["question_tok"].tolist()]
    sql = train_dataframe["sql"].tolist()
    table_ids = train_dataframe["table_id"].tolist()
    table_headers = [[x.lower() for x in train_table[tid]["header"]] for tid in table_ids]
    labels = parseSqlToLabels(sql)
    assert len(question_toks)==len(table_headers)
    
    keras_model_train_data = [{"query":question_toks[i],"header":table_headers[i]} for i in range(len(question_toks))]
    return keras_model_train_data, labels


In [8]:
def load_wikisql(path_wikisql, toy_model, toy_size, bert=False, no_w2i=False, no_hs_tok=False, aug=False):
    # Get data
    train_data, train_table = load_wikisql_data(path_wikisql, mode='train', toy_model=toy_model, toy_size=toy_size, no_hs_tok=no_hs_tok, aug=aug)
    dev_data, dev_table = load_wikisql_data(path_wikisql, mode='dev', toy_model=toy_model, toy_size=toy_size, no_hs_tok=no_hs_tok)


    # Get word vector
    if no_w2i:
        w2i, wemb = None, None
    else:
        w2i, wemb = load_w2i_wemb(path_wikisql, bert)


    return train_data, train_table, dev_data, dev_table, w2i, wemb


def load_wikisql_data(path_wikisql, mode='train', toy_model=False, toy_size=10, no_hs_tok=False, aug=False):
    """ Load training sets
    """
    if aug:
        mode = f"aug.{mode}"
        print('Augmented data is loaded!')

    path_sql = os.path.join(path_wikisql, mode+'_tok.jsonl')
    if no_hs_tok:
        path_table = os.path.join(path_wikisql, mode + '.tables.jsonl')
    else:
        path_table = os.path.join(path_wikisql, mode+'_tok.tables.jsonl')

    data = []
    table = {}
    with open(path_sql) as f:
        for idx, line in enumerate(f):
            if toy_model and idx >= toy_size:
                break

            t1 = json.loads(line.strip())
            data.append(t1)

    with open(path_table) as f:
        for idx, line in enumerate(f):
            if toy_model and idx > toy_size:
                break

            t1 = json.loads(line.strip())
            table[t1['id']] = t1

    return data, table


## FORMATTING SQL LABELS

In [7]:
def get_g(sql_i):
    """ for backward compatibility, separated with get_g"""
    g_sc = []
    g_sa = []
    g_wn = []
    g_wc = []
    g_wo = []
    g_wv = []
    for b, psql_i1 in enumerate(sql_i):
        g_sc.append( psql_i1["sel"] )
        g_sa.append( psql_i1["agg"])

        conds = psql_i1['conds']
        if not psql_i1["agg"] < 0:
            g_wn.append( len( conds ) )
            g_wc.append( get_wc1(conds) )
            g_wo.append( get_wo1(conds) )
            g_wv.append( get_wv1(conds) )
        else:
            raise EnvironmentError
    return g_sc, g_sa, g_wn, g_wc, g_wo, g_wv

def padWhereConditions(clause_list):
    b = np.zeros([len(clause_list),len(max(clause_list,key = lambda x: len(x)))])
    b[:] = -1
    for i,j in enumerate(clause_list):
        b[i][0:len(j)] = j
    return(b)
def multihotEncodeWhereThings(where_things):
    max_where_things = len(set([x for y in where_things for x in y]))
    where_things_multihot = np.zeros([len(where_things),max_where_things])
    for i, where_things_locs in enumerate(where_things):
        where_things_multihot[i][where_things_locs] = 1
        
    return where_things_multihot
    
def parseSqlToLabels(sql): 
    #return sql
    #return [random.randint(0, 1) for x in sql]
    sc,sa,wn,wc,wo,wv = get_g(sql)
    wc,wo = multihotEncodeWhereThings(wc),multihotEncodeWhereThings(wo)
    return np.array(sc),np.array(sa),np.array(wn),wc,wo

In [143]:
training_data, train_labels = makeTrainingData(use_reduced_set=True,sample_size=2000)
sc,sa,wn,wc,wo = train_labels

In [19]:
wc.shape,wo.shape

((2000, 13), (2000, 3))